## Face Alignment & Normalization (내 사진 100장)

In [1]:
import numpy as np
import cv2

def rect_to_bb(rect):
	# take a bounding predicted by dlib and convert it
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
	# return a tuple of (x, y, w, h)
	return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
	# initialize the list of (x, y)-coordinates
	coords = np.zeros((68, 2), dtype=dtype)
	# loop over the 68 facial landmarks and convert them
	# to a 2-tuple of (x, y)-coordinates
	for i in range(0, 68):
		coords[i] = (shape.part(i).x, shape.part(i).y)
	# return the list of (x, y)-coordinates
	return coords

from collections import OrderedDict

# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions

#For dlib’s 68-point facial landmark detector:
FACIAL_LANDMARKS_68_IDXS = OrderedDict([
	("mouth", (48, 68)),
	("inner_mouth", (60, 68)),
	("right_eyebrow", (17, 22)),
	("left_eyebrow", (22, 27)),
	("right_eye", (36, 42)),
	("left_eye", (42, 48)),
	("nose", (27, 36)),
	("jaw", (0, 17))
])

#For dlib’s 5-point facial landmark detector:
FACIAL_LANDMARKS_5_IDXS = OrderedDict([
	("right_eye", (2, 3)),
	("left_eye", (0, 1)),
	("nose", (4))
])

# in order to support legacy code, we'll default the indexes to the
# 68-point model
FACIAL_LANDMARKS_IDXS = FACIAL_LANDMARKS_68_IDXS

In [2]:
class FaceAligner:
    def __init__(self, predictor, desiredLeftEye=(0.35, 0.35),
        desiredFaceWidth=256, desiredFaceHeight=None):
        # store the facial landmark predictor, desired output left
        # eye position, and desired output face width + height
        self.predictor = predictor # 얼굴 랜드마크 예측 모델
        self.desiredLeftEye = desiredLeftEye # 원하는 출력 왼쪽 눈 위치를 지정
        self.desiredFaceWidth = desiredFaceWidth # 원하는 얼굴을 픽셀 단위로 정의, 기본값 256픽셀
        self.desiredFaceHeight = desiredFaceHeight #  원하는 얼굴 높이 값을 픽셀 단위로 지정
        # if the desired face height is None, set it to be the
        # desired face width (normal behavior)
        if self.desiredFaceHeight is None:
            self.desiredFaceHeight = self.desiredFaceWidth # 정사각형 이미지로 출력


    def align(self, image, gray, rect):
        # image: The RGB input image/ gray: The grayscale input image/ rect: The bounding box rectangle produced by dlib’s HOG face detector
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = self.predictor(gray, rect)
        shape = shape_to_np(shape)
        # extract the left and right eye (x, y)-coordinates
        if (len(shape)==68):
          (lStart, lEnd) = FACIAL_LANDMARKS_68_IDXS["left_eye"]
          (rStart, rEnd) = FACIAL_LANDMARKS_68_IDXS["right_eye"]
        else:
          (lStart, lEnd) = FACIAL_LANDMARKS_5_IDXS["left_eye"]
          (rStart, rEnd) = FACIAL_LANDMARKS_5_IDXS["right_eye"]

        leftEyePts = shape[lStart:lEnd]
        rightEyePts = shape[rStart:rEnd]

        # 각 눈의 질량 중심을 계산: 각 눈의 모든 (x, y) 점을 평균하여 각 눈의 중심, 즉 질량 중심을 계산
        leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
        rightEyeCenter = rightEyePts.mean(axis=0).astype("int")

        # 눈 중심 사이의 각도를 계산
        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX)) - 180 # 얼굴 회전의 각도를 계산

        # compute the desired right eye x-coordinate based on the desired x-coordinate of the left eye
        # 왼쪽 눈의 원하는 x좌표를 기반으로 원하는 오른쪽 눈 x 좌표를 계산
        desiredRightEyeX = 1.0 - self.desiredLeftEye[0]
        # determine the scale of the new resulting image by taking
        # the ratio of the distance between eyes in the *current*
        # image to the ratio of distance between eyes in the
        # *desired* image
        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - self.desiredLeftEye[0])
        desiredDist *= self.desiredFaceWidth
        scale = desiredDist / dist


        # compute center (x, y)-coordinates (i.e., the median point)
        # between the two eyes in the input image
        eyesCenter = (int((leftEyeCenter[0] + rightEyeCenter[0]) // 2),
              int((leftEyeCenter[1] + rightEyeCenter[1]) // 2))
        # grab the rotation matrix for rotating and scaling the face
        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)
        # update the translation component of the matrix
        tX = self.desiredFaceWidth * 0.5
        tY = self.desiredFaceHeight * self.desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        # apply the affine transformation
        (w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
        output = cv2.warpAffine(image, M, (w, h),
            flags=cv2.INTER_CUBIC)
        # return the aligned face
        return output

In [3]:
import dlib
import imutils
from google.colab.patches import cv2_imshow
from collections import OrderedDict
import matplotlib.pyplot as plt

In [4]:
shape_predictor_path = "/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/shape_predictor_68_face_landmarks.dat"

In [9]:
import os
from imutils import paths

# Set the directory containing the images and the output directory
input_dir = '/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data'  # 입력 이미지 폴더 경로
output_dir = '/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces'  # 출력 폴더 경로
os.makedirs(output_dir, exist_ok=True)  # 출력 폴더가 없으면 생성

# Initialize dlib's face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)
fa = FaceAligner(predictor, desiredFaceWidth=256)

# Loop over the input images
imagePaths = list(paths.list_images(input_dir))  # 폴더 내 이미지 경로 가져오기
for (i, imagePath) in enumerate(imagePaths):
    print(f"[INFO] Processing image {i + 1}/{len(imagePaths)}: {imagePath}")

    # Load the image, resize it, and convert it to grayscale
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=800)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image
    rects = detector(gray, 2)

    for rect in rects:
        # Align the face using facial landmarks
        faceAligned = fa.align(image, gray, rect)

        # Construct the new file name
        output_filename = f"aligned_{i + 1:03d}.jpg"  # 'aligned_001.jpg', 'aligned_002.jpg', ...
        output_path = os.path.join(output_dir, output_filename)

        # Save the aligned face
        cv2.imwrite(output_path, faceAligned)

print("[INFO] Face alignment complete.")

[INFO] Processing image 1/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data/IMG_3516.JPG
[INFO] Processing image 2/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data/IMG_2812.JPG
[INFO] Processing image 3/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data/IMG_4132.JPG
[INFO] Processing image 4/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data/IMG_2661.JPG
[INFO] Processing image 5/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data/IMG_3438.JPG
[INFO] Processing image 6/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data/F00EE39E-E194-4AB1-B5DC-C8E6EEE632EF.jpg
[INFO] Processing image 7/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data/EB8AF79A-3EAC-4FBC-9F56-6738785888B4.jpg
[INFO] Processing image 8/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data/5F423B94-5EEC-4EC5-A992-DB095AAD1165.jpg
[INF

### 사진 하나가 잘 정렬이 안돼서 추가로 하나 더 넣음

In [12]:
# load the input image, resize it, and convert it to grayscale
input_image_path = '/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data/증명사진.jpg'  # 처리할 단일 이미지 경로

# Load the input image, resize it, and convert it to grayscale
image = cv2.imread(input_image_path)  # 경로 변수 input_image_path 사용
if image is None:
    raise ValueError(f"[ERROR] Image not found at {input_image_path}")
image = imutils.resize(image, width=800)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the grayscale image
rects = detector(gray, 2)

# If faces are detected, align the first face
if len(rects) > 0:
    rect = rects[0]  # 첫 번째 얼굴만 처리
    faceAligned = fa.align(image, gray, rect)

    # Save the aligned face
    output_filename = "aligned_014.jpg"  # 출력 파일 이름
    output_path = os.path.join(output_dir, output_filename)
    cv2.imwrite(output_path, faceAligned)

    print(f"[INFO] Aligned face saved to {output_path}")
else:
    print("[INFO] No faces detected in the image.")

[INFO] Aligned face saved to /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces/aligned_014.jpg


## Bounding Box로 크롭하여 저장 (112*112 사이즈)

In [14]:
# Define paths
aligned_dir = '/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces'  # 정렬된 얼굴이 저장된 폴더
cropped_dir = '/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/cropped_faces'  # 크롭된 얼굴을 저장할 폴더
os.makedirs(cropped_dir, exist_ok=True)  # 출력 폴더 생성

# Desired output size
crop_size = 112

# Loop over the aligned images
aligned_image_paths = list(paths.list_images(aligned_dir))
for (i, aligned_image_path) in enumerate(aligned_image_paths):
    print(f"[INFO] Processing aligned image {i + 1}/{len(aligned_image_paths)}: {aligned_image_path}")

    # Load the aligned image
    image = cv2.imread(aligned_image_path)

    # Get the center of the image
    (h, w) = image.shape[:2]
    centerX, centerY = w // 2, h // 2

    # Compute the bounding box coordinates for the crop
    startX = max(0, centerX - crop_size // 2)
    startY = max(0, centerY - crop_size // 2)
    endX = min(w, centerX + crop_size // 2)
    endY = min(h, centerY + crop_size // 2)

    # Crop the image
    cropped_face = image[startY:endY, startX:endX]

    # Resize to 112x112 if not already the right size
    cropped_face = cv2.resize(cropped_face, (crop_size, crop_size), interpolation=cv2.INTER_AREA)

    # Save the cropped face
    cropped_filename = f"WSH{i + 1}.jpg"
    cropped_path = os.path.join(cropped_dir, cropped_filename)
    cv2.imwrite(cropped_path, cropped_face)

print("[INFO] Cropping and resizing complete.")

[INFO] Processing aligned image 1/89: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces/aligned_001.jpg
[INFO] Processing aligned image 2/89: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces/aligned_002.jpg
[INFO] Processing aligned image 3/89: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces/aligned_003.jpg
[INFO] Processing aligned image 4/89: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces/aligned_004.jpg
[INFO] Processing aligned image 5/89: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces/aligned_005.jpg
[INFO] Processing aligned image 6/89: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces/aligned_009.jpg
[INFO] Processing aligned image 7/89: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces/aligned_010.jpg
[INFO] Processing aligned image 8/89: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligne

### 89장밖에 안돼서 다시 추가로 정렬부터..

In [17]:
# Set the directory containing the images and the output directory
input_dir = '/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data1'  # 입력 이미지 폴더 경로
output_dir = '/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces1'  # 출력 폴더 경로
os.makedirs(output_dir, exist_ok=True)  # 출력 폴더가 없으면 생성

# Initialize dlib's face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)
fa = FaceAligner(predictor, desiredFaceWidth=256)

# Loop over the input images
imagePaths = list(paths.list_images(input_dir))  # 폴더 내 이미지 경로 가져오기
for (i, imagePath) in enumerate(imagePaths):
    print(f"[INFO] Processing image {i + 1}/{len(imagePaths)}: {imagePath}")

    # Load the image, resize it, and convert it to grayscale
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=800)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image
    rects = detector(gray, 2)

    for rect in rects:
        # Align the face using facial landmarks
        faceAligned = fa.align(image, gray, rect)

        # Construct the new file name
        output_filename = f"aligned_{i + 1:03d}.jpg"  # 'aligned_001.jpg', 'aligned_002.jpg', ...
        output_path = os.path.join(output_dir, output_filename)

        # Save the aligned face
        cv2.imwrite(output_path, faceAligned)

print("[INFO] Face alignment complete.")

[INFO] Processing image 1/19: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data1/beauty_1585711162357.jpeg
[INFO] Processing image 2/19: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data1/beauty_1594099731114.jpeg
[INFO] Processing image 3/19: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data1/beauty_1582711051674.jpeg
[INFO] Processing image 4/19: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data1/beauty_1584092738902.jpeg
[INFO] Processing image 5/19: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data1/beauty_1580955894583.jpeg
[INFO] Processing image 6/19: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data1/beauty_1580955887600.jpeg
[INFO] Processing image 7/19: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data1/beauty_1594099753978.jpeg
[INFO] Processing image 8/19: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/Face_data1/beauty_158

In [18]:
# Define paths
aligned_dir = '/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces1'  # 정렬된 얼굴이 저장된 폴더
cropped_dir = '/content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/cropped_faces'  # 크롭된 얼굴을 저장할 폴더
os.makedirs(cropped_dir, exist_ok=True)  # 출력 폴더 생성

# Desired output size
crop_size = 112

# Loop over the aligned images
aligned_image_paths = list(paths.list_images(aligned_dir))
for (i, aligned_image_path) in enumerate(aligned_image_paths):
    print(f"[INFO] Processing aligned image {i + 1}/{len(aligned_image_paths)}: {aligned_image_path}")

    # Load the aligned image
    image = cv2.imread(aligned_image_path)

    # Get the center of the image
    (h, w) = image.shape[:2]
    centerX, centerY = w // 2, h // 2

    # Compute the bounding box coordinates for the crop
    startX = max(0, centerX - crop_size // 2)
    startY = max(0, centerY - crop_size // 2)
    endX = min(w, centerX + crop_size // 2)
    endY = min(h, centerY + crop_size // 2)

    # Crop the image
    cropped_face = image[startY:endY, startX:endX]

    # Resize to 112x112 if not already the right size
    cropped_face = cv2.resize(cropped_face, (crop_size, crop_size), interpolation=cv2.INTER_AREA)

    # Save the cropped face
    cropped_filename = f"WSH{i + 90}.jpg"
    cropped_path = os.path.join(cropped_dir, cropped_filename)
    cv2.imwrite(cropped_path, cropped_face)

print("[INFO] Cropping and resizing complete.")

[INFO] Processing aligned image 1/18: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces1/aligned_014.jpg
[INFO] Processing aligned image 2/18: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces1/aligned_001.jpg
[INFO] Processing aligned image 3/18: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces1/aligned_002.jpg
[INFO] Processing aligned image 4/18: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces1/aligned_003.jpg
[INFO] Processing aligned image 5/18: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces1/aligned_004.jpg
[INFO] Processing aligned image 6/18: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces1/aligned_005.jpg
[INFO] Processing aligned image 7/18: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/aligned_faces1/aligned_006.jpg
[INFO] Processing aligned image 8/18: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트

### 몇개 골라내서 100장 사진 전처리 완!

### 크롭된 사진 크기가 모두 112*112인지 확인

In [19]:
# Loop over the cropped images
cropped_image_paths = list(paths.list_images(cropped_dir))
for (i, cropped_image_path) in enumerate(cropped_image_paths):
    print(f"[INFO] Processing cropped image {i + 1}/{len(cropped_image_paths)}: {cropped_image_path}")

    # Load the cropped image
    image = cv2.imread(cropped_image_path)

    # Check if the image is loaded successfully
    if image is None:
        print(f"[ERROR] Failed to load image {cropped_image_path}")
        continue  # Skip to the next image if loading fails

    # Get the size of the image (height, width, channels)
    (h, w) = image.shape[:2]

    # Print the size of the image
    print(f"[INFO] Cropped image size: {w}x{h}")

print("[INFO] Cropped image size checking complete.")

[INFO] Processing cropped image 1/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/cropped_faces/WSH1.jpg
[INFO] Cropped image size: 112x112
[INFO] Processing cropped image 2/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/cropped_faces/WSH2.jpg
[INFO] Cropped image size: 112x112
[INFO] Processing cropped image 3/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/cropped_faces/WSH3.jpg
[INFO] Cropped image size: 112x112
[INFO] Processing cropped image 4/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/cropped_faces/WSH4.jpg
[INFO] Cropped image size: 112x112
[INFO] Processing cropped image 5/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/cropped_faces/WSH5.jpg
[INFO] Cropped image size: 112x112
[INFO] Processing cropped image 6/100: /content/drive/MyDrive/Colab Notebooks/DAT 4th/팀프로젝트/cropped_faces/WSH6.jpg
[INFO] Cropped image size: 112x112
[INFO] Processing cropped image 7/100: /content/drive/MyDr